In [ ]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

import openai
from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY



In [50]:
from LLMSearch.AnswerBot import AnswerBot,parse_answer
from settings.key import GPT_API_KEY, DEEPL_API_KEY
from LLMSearch.GPTQuery import GPTQuery
from LLMSearch.BM25DB import BM25DB
from LLMSearch.SQLTextDB import SQLTextDB
from LLMSearch.Embedding.SBERTFineTuneEmbedding import SBERTFineTuneEmbedding
from LLMSearch.VectorSearcher import VectorSearcher

embedder=SBERTFineTuneEmbedding()
searcher=VectorSearcher(embedder)


#bm=BM25DB(initiate=True)
#searcher=SQLTextDB()

# initiate bot module
bot1 = AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=searcher,
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot2=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=SQLTextDB(), 
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot3=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=BM25DB(initiate=True),
                DEEPL_API_KEY=DEEPL_API_KEY,   
                )

INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
bot1.index_documents(initiate=False)
bot2.index_documents(initiate=False)
bot3.index_documents(initiate=False)

100%|██████████| 359/359 [00:00<00:00, 19227.15it/s]


  0%|          | 0/359 [00:00<?, ?it/s]

In [46]:
query="先生の業績は?"
query="先生の趣味は?"
query="藤田正博先生の業績は?"

related_documents=[]
for bot in [bot1,bot2]:
    related_documents+=bot.search_related_documents(query, k=20)

ans={}
ans["answer"]=""
ans["context"]=related_documents
str_ans=parse_answer(ans)
print(str_ans)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]



- Reference 1 
http://localhost:8099/UserData/original/text/kobunshi_ronbunshu/koron_68_12_68_12_767.txt 
Similarity 97.4901123046875
https://www.jstage.jst.go.jp/article/koron/68/12/68/12/767/_pdf/-char/ja
高分子論文集
7671お茶の水女子大学大学院人間文化 
***
- Reference 2 
http://localhost:8099/UserData/original/text/kobunshi_ronbunshu/koron_69_12_69_671.txt 
Similarity 89.78971099853516
https://www.jstage.jst.go.jp/article/koron/69/12/69/671/_pdf/-char/ja
高分子論文集
高分子論文集特集号および高分子科学・工学のニュ 
***
- Reference 3 
http://localhost:8099/UserData/original/text/kobunshi_ronbunshu/koron_69_12_69_671.txt 
Similarity 89.08512878417969
https://www.jstage.jst.go.jp/article/koron/69/12/69/671/_pdf/-char/ja
高分子論文集
高分子論文集特集号および高分子科学・工学のニュ 
***
- Reference 4 
http://localhost:8099/UserData/original/text/kobunshi_ronbunshu/koron_69_12_69_2012-0078.txt 
Similarity 87.38018798828125
https://www.jstage.jst.go.jp/article/koron/69/12/69_2012-0078/_pdf/-char/ja
高分子論文集
高分子学会・高分子論文集の思い出田 
***
- Reference 5 
http://localhost:8099/U

In [47]:
import json
import numpy as np

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return super(MyEncoder, self).default(obj)

#related documentsから､重複を除く
#related_documents=list(set(related_documents))
list_of_json = [json.dumps(d, sort_keys=True,cls=MyEncoder) for d in related_documents]

related_documents= list(map(json.loads, set(list_of_json)))
len(related_documents)

40

In [43]:
#related_documentを5こずつに分割
related_documents_list=[related_documents[i:i+5] for i in range(0,len(related_documents),5)]
#related_documents_list[1]

In [45]:
for related_documents in related_documents_list:
    response=bot1.ask(query,context_list= related_documents,stream=True)

    ans=response["answer"]
    for chunk in ans:
        next = chunk['choices'][0]['delta'].get('content', '')
        print(next, end='')

    print("\n")

The given context does not provide enough information to accurately answer the question about the achievements of 藤田先生 (Professor Fujita).

There is no information about a "Dr. Fujita" or any of their achievements in the given text.

There is no specific information about Dr. Fujita's achievements in the given context.

文脈から読み取れる情報に基づいて回答すると、藤田先生の業績については特定されていません。ただし、藤田先生の名前はいくつかの文献に登場しており、高分子科学や触媒に関する研究や論文に関わっている可能性があります。ただし、正確な情報を確認するためには、詳細な検索が必要となるかもしれません。

The given context doesn't provide information about which "藤田先生" is being referred to. Therefore, it's not possible to answer this question accurately.



Exception ignored in: <function tqdm.__del__ at 0x7f5d82ffd630>
Traceback (most recent call last):
  File "/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:2                                                                              │
│                                                                                                  │
│    1 for related_documents in related_documents_list:                                            │
│ ❱  2 │   response=bot1.ask(query,context_list= related_documents,stream=True)                    │
│    3 │                                                                                           │
│    4 │   ans=response["answer"]                                                                  │
│    5 │   for chunk in ans:                                                                       │
│                                                                                                  │
│ /media/kh/python/2023/20230427rwkvGPT/LLMSearch/AnswerBot.py:69 in ask                           │
│                                                                                                  │
│   66 │   │   │   context_list = self.search_related_documents(question, k, pad)                  │
│   67 │   │                                                                                       │
│   68 │   │   k = min(k, len(context_list))                                                       │
│ ❱ 69 │   │   ans = self.query_module.reference_ask(                                              │
│   70 │   │   │   question, context_list, k, stream=stream)                                       │
│   71 │   │                                                                                       │
│   72 │   │   if Ja_to_En:                                                                        │
│                                                                                                  │
│ /media/kh/python/2023/20230427rwkvGPT/LLMSearch/GPTQuery.py:38 in reference_ask                  │
│                                                                                                  │
│   35 │   │   context_text = context_text[:ref_max_length]                                        │
│   36 │   │                                                                                       │
│   37 │   │   res_dict = {}                                                                       │
│ ❱ 38 │   │   res_dict["answer"] = self.ask_gpt(query, context_text, stream=stream)               │
│   39 │   │   res_dict["context"] = context_list[:k]                                              │
│   40 │   │                                                                                       │
│   41 │   │   return res_dict                                                                     │
│                                                                                                  │
│ /media/kh/python/2023/20230427rwkvGPT/LLMSearch/GPTQuery.py:20 in ask_gpt                        │
│                                                                                                  │
│   17 │   │   │   │   {"role": "user", "content": query},                                         │
│   18 │   │   │   ]                                                                               │
│   19 │   │                                                                                       │
│ ❱ 20 │   │   response = openai.ChatCompletion.create(                                            │
│   21 │   │   │   model=self.model,                                                               │
│   22 │   │   │   messages=messages,                                                              │
│   23 │   │   │   stream=stream,                                                                  │
│                                                                                                  │
│ /home/kh/anaconda3/envs/lora/lib/python3.10/site-packages/o